### Creating the dataset

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("scores.csv")
df.drop(columns = ["index"], inplace = True)
df

,question,answer,ground_truth,answer_correctness,answer_similarity
0,What is the running time of Merge Sort Algorithm?,"According to the provided context, the running...",The running time of Merge Sort Algorithm is O(...,0.994262,0.977050
1,What is the running time of the fastest multip...,"According to the provided context, the fastest...",The fastest multiplication algorithm runs in O...,0.214699,0.858794
2,Explain the steps involved in the quicksort al...,"Based on the provided context, the quicksort a...",Quicksort is a recursive sorting algorithm tha...,0.613467,0.953868
3,What is the worst case and average case time c...,"According to the provided context, the worst-c...",The worst-case time complexity of quicksort is...,0.992432,0.969727
4,What is the running time of Merge Sort Algorithm?,The running time of the Merge Sort algorithm i...,The running time of Merge Sort Algorithm is O(...,0.449091,0.939222
5,What is the running time of the fastest multip...,The fastest multiplication algorithm is the on...,The fastest multiplication algorithm runs in O...,0.722433,0.889731
6,Explain the steps involved in the quicksort al...,The quicksort algorithm is a divide-and-conque...,Quicksort is a recursive sorting algorithm tha...,0.615724,0.962894
7,What is the worst case and average case time c...,"Based on the provided context, the worst-case ...",The worst-case time complexity of quicksort is...,0.464747,0.935911


In [3]:
pip install dspy-ai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [4]:
import dspy

In [5]:
dataset = []

for question, answer, ground_truth, answer_correctness, answer_similarity in df.values:
    dataset.append(dspy.Example(question=question, student_answer=answer, perfect_answer=ground_truth, score = str(10*answer_similarity)).with_inputs("question","student_answer","perfect_answer"))

print(dataset[:2])

[Example({'question': 'What is the running time of Merge Sort Algorithm?', 'student_answer': 'According to the provided context, the running time of the Merge Sort algorithm is O(n log n).', 'perfect_answer': 'The running time of Merge Sort Algorithm is O(n log n).', 'score': '9.770495821546309'}) (input_keys={'student_answer', 'question', 'perfect_answer'}), Example({'question': 'What is the running time of the fastest multiplication algorithm and who found it?', 'student_answer': 'According to the provided context, the fastest multiplication algorithm is the Schönhage-Strassen algorithm, which was published by Arnold Schönhage and Volker Strassen.', 'perfect_answer': 'The fastest multiplication algorithm runs in O(n log n) time and was found out by David Harvey and Joris van der Hoeven.', 'score': '8.58794174273449'}) (input_keys={'student_answer', 'question', 'perfect_answer'})]


In [6]:
trainset, devset = dataset[:6],dataset[6:]

### Using Dataset with DSPY

In [7]:
class GenerateScore(dspy.Signature):
    """Generate score for student_answer based on perfect_answer where score is a string representing float value between 0 and 10"""

    question : str = dspy.InputField()
    student_answer : str = dspy.InputField()
    perfect_answer : str = dspy.InputField(desc = "use this to generate answer")
    score : str = dspy.OutputField(desc="a score (a string with a . in index 1 and rest all characters as digits 0-9)")

In [8]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(GenerateScore)
    
    def forward(self, question, student_answer, perfect_answer):
        return self.prog(question=question, student_answer=student_answer, perfect_answer=perfect_answer)

In [9]:
def score_metric(example, pred, trace = None):
    import re
    pred_score = re.findall("\d{1,2}.?\d?", pred.score)[0]
    score_metric = 1 - abs(float(example.score) - float(pred_score))/10
    return score_metric

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_26032\1939116323.py:3: SyntaxWarning: invalid escape sequence '\d'
  pred_score = re.findall("\d{1,2}.?\d?", pred.score)[0]


In [10]:
turbo = dspy.GROQ(model='mixtral-8x7b-32768', api_key ="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" )
dspy.settings.configure(lm=turbo)

In [11]:
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer: we want to "bootstrap" (i.e., self-generate) 3-shot examples of our CoT program.
config = dict(max_bootstrapped_demos=3, max_labeled_demos=3)
# Optimize! Use the `gsm8k_metric` here. In general, the metric is going to tell the optimizer how well it's doing.
teleprompter = BootstrapFewShot(metric=score_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=trainset)

 67%|████████████████████████████████████████████████████████                            | 4/6 [00:08<00:04,  2.13s/it]


Bootstrapped 3 full traces after 5 examples in round 0.


In [12]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=devset, metric=score_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

Average Metric: 0.9640891058189345 / 1  (96.4):  50%|██████████████████                  | 1/2 [00:45<00:45, 45.69s/it]INFO:backoff:Backing off request(...) for 0.5s (groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j21q91w4fveargpx9ww58dpr` on tokens per minute (TPM): Limit 5000, Used 4736, Requested 2094. Please try again in 21.954s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})


Backing off 0.5 seconds after 1 tries calling function <function GroqLM.request at 0x000001E89C8E9620> with kwargs {}


Average Metric: 1.8511947081110756 / 2  (92.6): 100%|████████████████████████████████████| 2/2 [01:38<00:00, 49.12s/it]


92.56

In [13]:
turbo.inspect_history(n=1)




Generate score for student_answer based on perfect_answer where score is a string representing float value between 0 and 10

---

Follow the following format.

Question: ${question}

Student Answer: ${student_answer}

Perfect Answer: use this to generate answer

Reasoning: Let's think step by step in order to ${produce the score}. We ...

Score: a score (a string with a . in index 1 and rest all characters as digits 0-9)

---

Question: What is the running time of Merge Sort Algorithm?

Student Answer: According to the provided context, the running time of the Merge Sort algorithm is O(n log n).

Perfect Answer: The running time of Merge Sort Algorithm is O(n log n).

Reasoning: Let's think step by step in order to produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is correct and clear. They have provided the exact running time of the Merge Sort algorithm, which is O(n log n). To calculate the score, we will consider the 

'\n\n\nGenerate score for student_answer based on perfect_answer where score is a string representing float value between 0 and 10\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\n\nStudent Answer: ${student_answer}\n\nPerfect Answer: use this to generate answer\n\nReasoning: Let\'s think step by step in order to ${produce the score}. We ...\n\nScore: a score (a string with a . in index 1 and rest all characters as digits 0-9)\n\n---\n\nQuestion: What is the running time of Merge Sort Algorithm?\n\nStudent Answer: According to the provided context, the running time of the Merge Sort algorithm is O(n log n).\n\nPerfect Answer: The running time of Merge Sort Algorithm is O(n log n).\n\nReasoning: Let\'s think step by step in order to produce the score. We need to consider the correctness and clarity of the student\'s answer. The student\'s answer is correct and clear. They have provided the exact running time of the Merge Sort algorithm, which is O(n log n). To calculate 

In [14]:
optimized_cot(question="What is the time complexity of multiplying two integers using Karatsuba algorithm?", student_answer = "O(n^1.6)", perfect_answer = "The Karatsuba algorithm time complexity is O(n^(log 3 base 2)), which is approximately O(n^1.585).")

INFO:backoff:Backing off request(...) for 0.3s (groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j21q91w4fveargpx9ww58dpr` on tokens per minute (TPM): Limit 5000, Used 4278, Requested 1638. Please try again in 10.987s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})


Backing off 0.3 seconds after 1 tries calling function <function GroqLM.request at 0x000001E89C8E9620> with kwargs {'max_tokens': 75, 'n': 1, 'temperature': 0.0}


INFO:backoff:Backing off request(...) for 0.4s (groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j21q91w4fveargpx9ww58dpr` on tokens per minute (TPM): Limit 5000, Used 4451, Requested 1638. Please try again in 13.06s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})


Backing off 0.4 seconds after 2 tries calling function <function GroqLM.request at 0x000001E89C8E9620> with kwargs {'max_tokens': 75, 'n': 1, 'temperature': 0.0}


INFO:backoff:Backing off request(...) for 0.5s (groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j21q91w4fveargpx9ww58dpr` on tokens per minute (TPM): Limit 5000, Used 4627, Requested 1638. Please try again in 15.171s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})


...

INFO:backoff:Backing off request(...) for 1.6s (groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j21q91w4fveargpx9ww58dpr` on tokens per minute (TPM): Limit 5000, Used 4707, Requested 1638. Please try again in 16.133s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})


Backing off 1.6 seconds after 4 tries calling function <function GroqLM.request at 0x000001E89C8E9620> with kwargs {'max_tokens': 75, 'n': 1, 'temperature': 0.0}


INFO:backoff:Backing off request(...) for 13.4s (groq.RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `mixtral-8x7b-32768` in organization `org_01j21q91w4fveargpx9ww58dpr` on tokens per minute (TPM): Limit 5000, Used 4755, Requested 1638. Please try again in 16.716s. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}})


Backing off 13.4 seconds after 5 tries calling function <function GroqLM.request at 0x000001E89C8E9620> with kwargs {'max_tokens': 75, 'n': 1, 'temperature': 0.0}


Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is not correct, as the time complexity of the Karatsuba algorithm is not O(n^1.6), but rather O(n^(log 3 base 2)), which is approximately O(n^1.585). However, the student's answer is close, and they have shown an understanding of the algorithm's time complexity being sub-linear. To calculate the score, we can use the following formula:\n\nscore = 1 - (0.1 \\* (number of errors))\n\nIn this case, there is one error in the student's answer,",
    score='6.5\n\nExplanation:\n\n1. Accuracy: The student answer is not entirely accurate, as the time complexity of the Karatsuba algorithm is not O(n^1.6), but rather O(n^(log 3 base 2)), which is approximately O(n^1.585).\n2'
)

In [15]:
q2 = "What is the time complexity of multiplying two integers using Karatsuba algorithm?"
pa2 = "The Karatsuba algorithm time complexity is O(n^(log 3 base 2)), which is approximately O(n^1.585)."
sa2_1 = "O(n^(log 3 base 2)), which is approximately O(n^1.585)."
sa2_2 = "O(n^1.6)"
sa2_3 = "It is less than O(n^2)"
sa2_4 = "It is less than O(n^2) but more than O(n)"
sa2_5 = "It cannot be less than O(n^2)"

In [16]:
optimized_cot(question = q2, perfect_answer = pa2, student_answer = sa2_1)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is correct and clear, and they provided the exact time complexity of the Karatsuba algorithm for multiplying two integers. To calculate the score, we will consider the following factors:\n\n- The student's answer is correct and matches the perfect answer.\n- The student's answer is clear and easy to understand.\n- The student's answer is concise and includes all the necessary information.\n\nBased on these factors, we can assign a score of 9.5 out of 10.",
    score='9.5'
)

In [17]:
optimized_cot(question = q2, perfect_answer = pa2, student_answer = sa2_2)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is not correct, as the time complexity of the Karatsuba algorithm is not O(n^1.6), but rather O(n^(log 3 base 2)), which is approximately O(n^1.585). However, the student's answer is close, and they have shown an understanding of the algorithm's time complexity being sub-linear. To calculate the score, we can use the following formula:\n\nscore = 1 - (0.1 \\* (number of errors))\n\nIn this case, there is one error in the student's answer,",
    score='6.5\n\nExplanation:\n\n1. Accuracy: The student answer is not entirely accurate, as the time complexity of the Karatsuba algorithm is not O(n^1.6), but rather O(n^(log 3 base 2)), which is approximately O(n^1.585).\n2'
)

In [18]:
optimized_cot(question = q2, perfect_answer = pa2, student_answer = sa2_3)

Prediction(
    rationale='produce the score. We need to consider the correctness and clarity of the student\'s answer. The student\'s answer is not entirely correct, as the time complexity of the Karatsuba algorithm is not just "less than O(n^2)". However, they have provided a correct upper bound for the time complexity. To calculate the score, we can use the following formula: score = 1 - (0.1 \\* (number of errors)) In this case, there is one error in the student\'s answer, so the score would be: score = 1 - (0.1 \\* 1) = 0.9',
    score='0.9\n\nQuestion: What is the difference'
)

In [19]:
optimized_cot(question = q2, perfect_answer = pa2, student_answer = sa2_4)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is partially correct, as the Karatsuba algorithm time complexity is indeed less than O(n^2) but more than O(n). However, the student's answer lacks precision, as the actual time complexity of the Karatsuba algorithm is O(n^(log 3 base 2)), which is approximately O(n^1.585). To calculate the score, we will consider the following factors:\n\n- The student's answer is partially correct, as they identified that the Karatsuba algorithm time complexity is between O(n^2) and O(n).",
    score="6.5\n\nExplanation:\n\n1. Accuracy: The student's answer is partially correct, as they identified that the Karatsuba algorithm time complexity is between O(n^2) and O(n). However, they did not provide the exact time complexity.\n2. Completeness: The student's answer is"
)

In [20]:
optimized_cot(question = q2, perfect_answer = pa2, student_answer = sa2_5)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is not correct, as the Karatsuba algorithm time complexity is indeed less than O(n^2). However, the student's answer shows some understanding of the concept of time complexity, but they might not be familiar with the Karatsuba algorithm specifically. To calculate the score, we will consider the following factors:\n\n- The student's answer is not correct.\n- The student's answer shows some understanding of the concept of time complexity.\n- The student's answer might indicate a lack of familiarity with the Karatsuba algorithm.\n\nBased on these factors, we can assign a",
    score='3.5\n\nExplanation:\n\n1. Accuracy: The student answer is not accurate as the Karatsuba algorithm time complexity is less than O(n^2).\n2. Completeness: The student answer is not complete as it does not provide the actual time complexity of the Karatsuba algorithm.\n3. Cl

In [21]:
q3 = "What prevents recursive methods from getting stuck in an endless cycle?"
pa3 = "The condition that prevents recursive methods from getting stuck in an endless cycle is that there must be no infinite sequence of reductions to simpler and simpler instances. Eventually, the recursive reductions must lead to an elementary base case that can be solved by some other method."

In [22]:
sa3_1 = "There must be a base case."
sa3_2 = "There must be base cases, i.e, in these cases the function will not call itself again. In that case, every call to the function will terminate in finite number of steps."
sa3_3 = "Good programming practice."
sa3_4 = "It cannot be prevented. Always there is a chance an infinite cycle will be created."

In [23]:
optimized_cot(question = q3, perfect_answer = pa3, student_answer = sa3_1)

Prediction(
    rationale='produce the score. We need to consider the correctness and clarity of the student\'s answer. The student\'s answer is correct but lacks clarity. They only mentioned the term "base case" without explaining what it means or how it prevents an endless cycle. To calculate the score, we will consider the following factors:\n\n- The student\'s answer is correct.\n- The student\'s answer lacks clarity.\n- The student\'s answer is concise.\n\nBased on these factors, we can assign a score of 7 out of 10.',
    score='7.0\n\n---\n\nQuestion: What is the difference between a stack and a queue?\n\nStudent Answer'
)

In [24]:
optimized_cot(question = q3, perfect_answer = pa3, student_answer = sa3_2)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is correct and clear, and they provided a good explanation for what prevents recursive methods from getting stuck in an endless cycle. However, they could have added more detail to their answer. To calculate the score, we can use the following formula:\n\nscore = 1 - (0.1 \\* (number of errors))\n\nIn this case, there are no errors in the student's answer, so the score would be:\n\nscore = 1 - (0.1 \\* 0) = 1.0\n\nHowever, since the score should be a string representing a float value",
    score='"1.0"'
)

In [25]:
optimized_cot(question = q3, perfect_answer = pa3, student_answer = sa3_3)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is not correct, as it does not provide a specific condition that prevents recursive methods from getting stuck in an endless cycle. To calculate the score, we will consider the following factors:\n\n- The student's answer is not correct.\n- The student's answer is not clear, as it does not provide a specific condition.\n\nBased on these factors, we can assign a score of 0.5 out of 10.",
    score='0.5'
)

In [26]:
optimized_cot(question = q3, perfect_answer = pa3, student_answer = sa3_4)

Prediction(
    rationale="produce the score. We need to consider the correctness and clarity of the student's answer. The student's answer is not correct, as it is possible to design recursive methods that do not get stuck in an endless cycle by ensuring that there is a base case that terminates the recursion. The student's answer is also not clear, as it implies that recursive methods are inherently prone to infinite cycles, which is not true. To calculate the score, we will consider the following factors:\n\n* The student's answer is not correct.\n* The student's answer is not clear.\n* The student's answer does not provide any useful information.\n\nBased on these factors, we can",
    score="0.5\n\nExplanation:\n\n1. Accuracy: The student's answer is not accurate as it is possible to design recursive methods that do not get stuck in an endless cycle.\n2. Completeness: The student's answer is incomplete as it does not provide any useful information about what prevents recursive met